# Testbenches for the DC Analysis

The goal of this notebook is to compare the results from Yarf simulations to what is obtained using Xyce. There are several Xyce netlists in the ./data directory, along with the output data. They were developed with the help of Qucs-S. The netlists are recreated and simulated here using Yarf and the results between the engines are compared.

In [1]:
import sys
sys.path.append("..")

import logging
import numpy as np
import matplotlib.pyplot as plt

from Xyce import getXyceData
from yarf import Yarf

# Circuit 1

<img src="data/circuit1.png" alt="drawing" width="600"/>

In [2]:
y = Yarf("Circuit 1")

y.add_resistor('R1', 'n1', 'n2', 100)
y.add_resistor('R2', 'n2', 'gnd', 25)
y.add_resistor('R3', 'n3', 'n5', 50)
y.add_resistor('R4', 'n7', 'n8', 33)
y.add_resistor('R5', 'n8', 'n5', 66)
y.add_resistor('R6', 'n9', 'gnd', 50)
y.add_capacitor('C1', 'n4', 'n3', 1e-12)
y.add_inductor('L1', 'n4', 'n2', 1e-9)
y.add_inductor('L2', 'n5', 'gnd', 1e-9)

v1 = y.add_vdc('V1', 'n1', 'gnd', 10)
y.add_vdc('V2', 'n3', 'n2', 7)
y.add_idc('I1', 'n4', 'n1', 2)

y.add_vcvs('SRC1', 'n3', 'n7', 'n8', 'n5', 1.7)
y.add_ccvs('SRC2', 'n7', 'n9', 'gnd', 'n5', 12.4)

y.add_dc_analysis('DC1')

# get output from Xyce simulator
xyce = getXyceData('data/circuit1.prn')

# sweep of V1 DC voltage
vsweep = np.arange(1, 11, 1)
i = 0
for v in vsweep:
    # run dc analysis with new dc voltage
    v1.dc = v
    y.run('DC1')

    # get output data
    vn1 = y.get_voltage('DC1', 'n1')
    vn2 = y.get_voltage('DC1', 'n2')
    vn3 = y.get_voltage('DC1', 'n3')
    vn4 = y.get_voltage('DC1', 'n4')
    vn5 = y.get_voltage('DC1', 'n5')
    vn7 = y.get_voltage('DC1', 'n7')
    vn8 = y.get_voltage('DC1', 'n8')
    vn9 = y.get_voltage('DC1', 'n9')

    # compare outputs from Xyce and Yarf
    yarf_out = [vn1, vn2, vn3, vn4, vn5, vn7, vn8, vn9]
    xyce_out = xyce[1][i][1:]
    print(max(yarf_out - xyce_out))
    i = i + 1

2.8451578870658523e-08
4.497874783737643e-08
3.970909823181046e-08
4.5337861465100104e-08
1.6438470140656136e-08
2.7296451321490167e-08
3.074495680266409e-08
1.299849827773869e-08
4.505143635924469e-08
2.2046933167985117e-09
